In [ ]:
%%sh
cat > pi-mpi.c << EOF

#include <mpi.h>
#include <stdio.h>
#define NELTS 4 /* number of elements in the vector */
#define MASTER 0 /* id of the first process */
#define FROM_MASTER 1 /* a message type */
#define FROM_WORKER 2 /* a message type */
#define N 4





MPI_Status status;
int main(int argc, char *argv[]) {
  double v[NELTS], /* the vector to be summed */
  sum, /* the overall vector sum */
  ptSum; /* a partial sum received from a worker */
  int numProcs, /* number of processes in virtual machine */
  numWorkers,   /* number of worker processes */
  rank,          /* our rank */
  source,      /* rank of message source */
  dest,      /* rank of message destination */
  mtype,     /* message type */
  aveSz,     /* average partition size */
  extra,     /* number of left over elements */
  offset,    /* offset of partition in v */
  i,        /* simple counter */
  count;   /* the actual number of elements in a partition */


//Liczba wierszy i kolumn w macierzach A, B, C
double A[N][N],B[N][N],C[N][N];
int processCount, processId, slaveTaskCount ;

// MPI 
MPI_Init(&argc, &argv);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &numProcs);
numWorkers = numProcs-1;


/******* master process ***********/
if (rank == MASTER) {
  // Initialize the vector
  for (i=0; i<NELTS; i++)v[i]= 1;
  
  
  // send vector partitions to the workers
  aveSz = NELTS/numWorkers;
  extra = NELTS%numWorkers;
  offset = 0;
  mtype = FROM_MASTER;
  
  for (dest=1; dest<=numWorkers; dest++) { 
    // start at 1 not 0 (no master)
    count = (dest <= extra) ? aveSz+1 : aveSz;
    MPI_Send(&count,1,MPI_INT,dest,mtype, MPI_COMM_WORLD);
    MPI_Send(&v[offset],count,MPI_DOUBLE,dest,mtype,MPI_COMM_WORLD);
    printf("Master sent elements %d to %d to rank %d.\n", offset,offset+count,dest);
    offset+=count;
  }


    // Inicjalizacja  Maceierze Dwuwymiarowej 
    // Inicjalizacja macierzy A i B


      double start = MPI_Wtime();
      srand ( time(NULL) );
      
      for (int i = 0; i<N; i++) {
        for (int j = 0; j<N; j++) {
        A[i][j]= rand()%10;
        B[i][j]= rand()%10;
      }}
      
      //Wypisanie zawartości macierzy A i B
      printf("\n Mnozenie macierzy za pomoca MPI \n");
      printf("\nMacierz A\n\n");
      
      for (int i = 0; i<N; i++) {
        for (int j = 0; j<N; j++) {
        printf("%.0f\t", A[i][j]);
      }
    printf("\n");
  }
  printf("\nMacierz B\n\n");
  for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
  printf("%.0f\t", B[i][j]);
  }
  }


/* wait for results from workers and sum them */
sum=0;
mtype = FROM_WORKER;


for (i=1; i<=numWorkers; i++) { 
  // start at 1 not 0 (no master)
  source = i;
  MPI_Recv(&ptSum,1,MPI_DOUBLE,source,mtype,MPI_COMM_WORLD, &status);
  sum+=ptSum;
}
  printf("Vector sum is %lf.\n", sum);
}

 // end master
/************ worker processes *************/
if (rank > MASTER) {
// receive out partition from the master
mtype = FROM_MASTER;
source = MASTER;
MPI_Recv(&count,1,MPI_INT,source,mtype,MPI_COMM_WORLD,&status);
MPI_Recv(&v,count,MPI_DOUBLE,source,mtype,MPI_COMM_WORLD,&status);
// compute our partial sum
sum=0;
//for (i=0;i<count;i++)
 // sum+=v[i];  




 for(int o=0;o<count;o++)
 {
 	for(int p=0;p<count;p++)
 	{
 		sum =+  v[o] * A[o][p];
 	//	printf("[%d]=%.1f\t", p, s[p]);
 	}
 }
  
/***/ printf("Slave %d is sending partial sum %lf to master.\n",rank,sum);
// send our partial sum to the master
mtype = FROM_WORKER;
MPI_Send(&sum,1,MPI_DOUBLE,MASTER,mtype,MPI_COMM_WORLD);
} // end worker
MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 5 --allow-run-as-root a.out


Master sent elements 0 to 1 to rank 1.
Master sent elements 1 to 2 to rank 2.
Master sent elements 2 to 3 to rank 3.
Master sent elements 3 to 4 to rank 4.

 Mnozenie macierzy za pomoca MPI 

Macierz A

3	3	4	6	
1	0	7	9	
1	7	2	7	
Slave 3 is sending partial sum 0.000000 to master.
2	6	5	8	

Macierz B

Slave 1 is sending partial sum 0.000000 to master.
Slave 2 is sending partial sum 0.000000 to master.
Slave 4 is sending partial sum 0.000000 to master.
1	0	3	6	9	7	2	1	5	8	1	5	2	5	2	8	Vector sum is 0.000000.


pi-mpi.c: In function ‘main’:
pi-mpi.c:70:7: warning: implicit declaration of function ‘srand’; did you mean ‘scanf’? [-Wimplicit-function-declaration]
       srand ( time(NULL) );
       ^~~~~
       scanf
pi-mpi.c:70:15: warning: implicit declaration of function ‘time’ [-Wimplicit-function-declaration]
       srand ( time(NULL) );
               ^~~~
pi-mpi.c:74:18: warning: implicit declaration of function ‘rand’ [-Wimplicit-function-declaration]
         A[i][j]= rand()%10;
                  ^~~~
